In [1]:
# Initial scraping approach implemented at https://github.com/israel-dryer/Indeed-Job-Scraper/blob/master/indeed-tutorial.ipynb
import csv
from datetime import datetime
import requests
from bs4 import BeautifulSoup

# Exploring indeed.com page elements

In [2]:
def get_url(position, location):
  """Generate a url  from position and location """
  template = "https://www.indeed.com/jobs?q={}&l={}"
  url = template.format(position, location)
  return url

In [3]:
url = get_url('data scientist', 'bentonville ar')

In [4]:
response = requests.get(url)

In [5]:
response

<Response [200]>

In [6]:
response.reason

''

In [7]:
soup = BeautifulSoup(response.text, 'html.parser')

In [9]:
cards = soup.find_all('div', 'jobsearch-SerpJobCard')

In [10]:
len(cards)

15

In [11]:
card = cards[0]

In [12]:
atag = card.h2.a

In [13]:
job_title = atag.get('title')

In [14]:
job_url = 'https://www.indeed.com' + atag.get('href')

In [15]:
company = card.find('span', 'company').text.strip()

In [16]:
job_location = card.find('div', 'recJobLoc').get('data-rc-loc')

In [17]:
summary = card.find('div', 'summary').text.strip()

In [18]:
date = card.find('span', 'date').text

In [19]:
today = datetime.today().strftime('%Y-%m-%d')

In [20]:
try: 
  card.find('span', 'salarText').text.strip()
except AttributeError:
  job_salary = '' 

# Generalizing the model

In [21]:
def get_record(card):
  atag = card.h2.a
  job_title = atag.get('title')
  job_url = 'https://www.indeed.com' + atag.get('href')
  company = card.find('span', 'company').text.strip()
  job_location = card.find('div', 'recJobLoc').get('data-rc-loc')
  job_summary = card.find('div', 'summary').text.strip()
  post_date = card.find('span', 'date').text
  today = datetime.today().strftime('%Y-%m-%d')
  try: 
    card.find('span', 'salarText').text.strip()
  except AttributeError:
    job_salary = '' 
  
  record = {job_title, company, job_location, post_date, today, job_summary, job_salary, job_url}

  return record

In [22]:
records = []

for card in cards: 
  record = get_record(card)
  records.append(record)

In [212]:
records[0]

{'',
 '2021-03-26',
 '30+ days ago',
 'A successful candidate will manage data science projects including scoping, designing, and executing the vision with the help of more junior data scientists /…',
 'Lead Data Scientist',
 'TRANZACT',
 'United States',
 'https://www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0DmvEs89GDjygIsDT0FtjWiil-qnA5TS0Npqc8I6T5HHPj1qJcB6ts2RC9U8mqwlH31VRyGgFTjuk6MDSQy9dlxHVsLO1FyLeaovTNDU0XCRmuZoptVyQ-71WeG21w73lkklPyJrRMYwYxr_-SpOcX4VcXq5o9hFdkEJ6YMkrQ-SFzGNTaQyCYYYPfdK6oMPeezYlV4gc8rXJAmU2-2RXBUFuc8ZuyTblvBUnhWwM92fTiyRe8-tYKNKVryJzixYYCvIFPCD9n55YgD0T8WAaZLjFxPGDchk_ju8RA2YX1QwdAl7k-Hr8zrzaVqoJfwjZbju5u65fojTr-cNoLsygZWsnPCUdhcw3R1yMNp6isnyHb-DH-hrp_dNKDD4PhQq2-lN14O0RoCNq2811yThejlJ5D4Jryqk1nemMfvfvS1_3IR8aVecl1xn40KWM440BMTZyHjQLppJ-MH8JCuREJB&p=0&fvj=1&vjs=3'}

# Getting every page

In [23]:
while True: 
  try: 
    url = 'https://www.indeed.com' + soup.find('a', {'aria-label': 'Next'}).get('href')
  except AttributeError:
    break

  response = requests.get(url)
  soup = BeautifulSoup(response.text, 'html.parser')
  cards = soup.find_all('div', 'jobsearch-SerpJobCard')

  for card in cards:
    record = get_record(card)
    records.append(record)

# Putting it all together

In [24]:
def get_url(position, location):
    """Generate url from position and location"""
    template = 'https://www.indeed.com/jobs?q={}&l={}'
    position = position.replace(' ', '+')
    location = location.replace(' ', '+')
    url = template.format(position, location)
    return url


def get_record(card):
    """Extract job data from a single record"""
    
    job_title = card.h2.a.get('title')
    company = card.find('span', 'company').text.strip()
    job_location = card.find('div', 'recJobLoc').get('data-rc-loc')
    post_date = card.find('span', 'date').text
    today = datetime.today().strftime('%Y-%m-%d')
    summary = card.find('div', 'summary').text.strip().replace('\n', ' ')
    job_url = 'https://www.indeed.com' + card.h2.a.get('href')

    # this does not exists for all jobs, so handle the exceptions
    salary_tag = card.find('span', 'salaryText')
    if salary_tag:
        salary = salary_tag.text.strip()
    else:
        salary = ''  
        
    record = (job_title, company, job_location, post_date, today, summary, salary, job_url)
    return record


def main(position, location):
    """Run the main program routine"""
    records = []
    url = get_url(position, location)
    
    # extract the job data
    while True:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        cards = soup.find_all('div', 'jobsearch-SerpJobCard')
        for card in cards:
            record = get_record(card)
            records.append(record)
        try:
            url = 'https://www.indeed.com' + soup.find('a', {'aria-label': 'Next'}).get('href')
        except AttributeError:
            break
        
    # save the job data
    with open('results.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['JobTitle', 'Company', 'Location', 'PostDate', 'ExtractDate', 'Summary', 'Salary', 'JobUrl'])
        writer.writerows(records)

In [25]:
main('data scientist', 'bentonville, ar')

In [26]:
import pandas as pd

In [30]:
df = pd.read_csv('results.csv')

In [31]:
df.head(1000)

,JobTitle,Company,Location,PostDate,ExtractDate,Summary,Salary,JobUrl
0,Lead Data Scientist,TRANZACT,United States,Active 7 days ago,2021-03-26,A successful candidate will manage data scienc...,"$150,000 - $165,000 a year",https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...
1,Data Scientist,Walmart,"Bentonville, AR",13 days ago,2021-03-26,"Data science, machine learning, optimization m...",NaN,https://www.indeed.com/rc/clk?jk=106b31ad1f6f0...
2,Data Scientist II,JB Hunt,"Lowell, AR",30+ days ago,2021-03-26,Assists in analytics project scoping discussio...,NaN,https://www.indeed.com/rc/clk?jk=ddace5a665cbc...
3,Senior Data Scientist,"Tyson Foods, Inc.","Springdale, AR",8 days ago,2021-03-26,We are seeking experienced members to join the...,NaN,https://www.indeed.com/rc/clk?jk=a1ceb152e0e8a...
4,Senior Data Scientist,REPL Consulting Inc,"Bentonville, AR",30+ days ago,2021-03-26,Strong related academic background i.e. Master...,NaN,https://www.indeed.com/rc/clk?jk=52bc7adf99bc4...
5,Senior Data Scientist,Walmart,"Bentonville, AR",Just posted,2021-03-26,"Certificates earned in data mining, machine le...",NaN,https://www.indeed.com/rc/clk?jk=2a7b0fd5433e0...
6,Division Data Science Analyst,Arvest,"Bentonville, AR",8 days ago,2021-03-26,"Utilizes data discovery, diagnostic, predicati...",NaN,https://www.indeed.com/rc/clk?jk=9c4d4e7884bf3...
7,Senior Data Scientist- Sam's Club,Sam's Club,"Bentonville, AR",2 days ago,2021-03-26,You will use advanced analytics techniques to ...,NaN,https://www.indeed.com/rc/clk?jk=fd56b421f1df7...
8,Data Scientist - Nationwide Opportunities,"Amazon Web Services, Inc.","Fayetteville, AR",30+ days ago,2021-03-26,Experience in an ML engineer or data scientist...,NaN,https://www.indeed.com/rc/clk?jk=b60071419fecb...
9,Senior Data Analyst,UAMS Medical Center,"Fayetteville, AR",30+ days ago,2021-03-26,Master Degree in a relevant field such as scie...,NaN,https://www.indeed.com/rc/clk?jk=bcdafbc670093...
